# Convert csv to sql or db file

## rating file

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
%%time
csv_file_path='/content/drive/My Drive/Data/SQL/rating.csv'
data_df = pd.read_csv(csv_file_path)
data_df.shape

CPU times: user 14.7 s, sys: 1.61 s, total: 16.3 s
Wall time: 17.3 s


Convert to sql by sqlalchemy

In [ ]:
%%time
from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo=False)
data_df.to_sql('rating_new', con=engine,chunksize=1000# index=True, index_label='id', if_exists='replace'
            )

In [ ]:
#engine.execute("SELECT * FROM rating_new").fetchall()

Convert to db by sqplite3 - best method

In [ ]:
%%time
import pandas as pd
import numpy as np
import sqlite3

# try to read db with large chunksize
db = sqlite3.connect("/content/drive/My Drive/Data/SQL/rating.sqlite")
path='/content/drive/My Drive/Data/SQL/rating.csv'
# Load the CSV in chunks:
for c in pd.read_csv(path, chunksize=10000):
    c.to_sql("rating", db, if_exists="append")

CPU times: user 1min 37s, sys: 11.3 s, total: 1min 48s
Wall time: 3min 15s


In [ ]:
datapath='/content/drive/My Drive/Data/SQL/rating.sqlite'
#datapath='/content/rating.sqlite'
conn = sqlite3.connect(datapath)
cursor = conn.cursor()

cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
table_name=cursor.fetchall()
table_name=table_name[0][0]
print('table_name: ',table_name)

table_name:  rating


## orders file

In [ ]:
%%time
path='/content/drive/My Drive/Data/Instacart Market dataset/orders.csv' #103 mb
df_2=pd.read_csv(path)
print(df_2.shape)

(3421083, 7)
CPU times: user 1.6 s, sys: 275 ms, total: 1.87 s
Wall time: 3.74 s


In [ ]:
df_2.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [ ]:
%%time
from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo=False)
df_2.to_sql('/content/drive/My Drive/Data/SQL/order_new', con=engine,chunksize=10000# index=True, index_label='id', if_exists='replace'
            )

CPU times: user 35.9 s, sys: 1.6 s, total: 37.5 s
Wall time: 37.7 s


# SQL analysis

## Rating.sqlite (complete)

In [ ]:
import sqlite3
datapath='/content/drive/My Drive/Data/SQL/rating_db.sqlite'
conn = sqlite3.connect(datapath)
cursor = conn.cursor()

cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
table_name=cursor.fetchall()
table_name=table_name[0][0]
print('table_name: ',table_name)

table_name:  rating


In [ ]:
%%time
cursor.execute("SELECT count(*) FROM %s" %table_name)
values = cursor.fetchone()
print ('number of rows: ',values[0])

number of rows:  20000263


In [ ]:
limit=50
df = pd.read_sql("SELECT * FROM %s LIMIT %s" % (table_name,limit), conn)
print(df.shape)
df.head()

(50, 5)


,index,userId,movieId,rating,time
0,0,1,2,3.5,2005-04-02 23:53:47
1,1,1,29,3.5,2005-04-02 23:31:16
2,2,1,32,3.5,2005-04-02 23:33:39
3,3,1,47,3.5,2005-04-02 23:32:07
4,4,1,50,3.5,2005-04-02 23:29:40


In [ ]:
df = pd.read_sql("SELECT * FROM %s" % (table_name), conn)
df.shape

(20000263, 5)

In [ ]:
table_name

'rating'

In [ ]:
pd.read_sql("SELECT COUNT(*) FROM %s Where rating='3.5'" % (table_name), conn)

,count(*)
0,2200156


In [ ]:
df.rating.value_counts() # same as SQL query, but a lot of RAM required

4.0    5561926
3.0    4291193
5.0    2898660
3.5    2200156
4.5    1534824
2.0    1430997
2.5     883398
1.0     680732
1.5     279252
0.5     239125
Name: rating, dtype: int64

In [ ]:
pd.read_sql("SELECT DISTINCT(rating) FROM %s " % (table_name), conn)

,rating
0,3.5
1,4.0
2,3.0
3,4.5
4,5.0
5,2.0
6,1.0
7,2.5
8,0.5
9,1.5


In [ ]:
pd.read_sql("SELECT COUNT (DISTINCT rating) FROM rating ", conn)

In [ ]:
pd.read_sql("SELECT COUNT (DISTINCT movieId) FROM rating ", conn)

,Count (distinct movieId)
0,26744


In [ ]:
pd.read_sql("SELECT COUNT (DISTINCT userId) FROM rating ", conn)

,Count (distinct userId)
0,138493


In [ ]:
cursor.execute("SELECT Count (distinct userId) FROM rating ")
row = cursor.fetchone()
print(row)

(138493,)


In [ ]:
cursor.execute("SELECT rating,count(*) FROM rating GROUP BY rating ")
row = cursor.fetchall()
row

[(0.5, 239125),
 (1.0, 680732),
 (1.5, 279252),
 (2.0, 1430997),
 (2.5, 883398),
 (3.0, 4291193),
 (3.5, 2200156),
 (4.0, 5561926),
 (4.5, 1534824),
 (5.0, 2898660)]

In [ ]:
cursor.execute("SELECT userId FROM rating WHERE rating='4.0' LIMIT 200 ")
row = cursor.fetchall()
print(row)

[(1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (2,), (3,), (3,), (3,), (3,), (3,), (3,), (3,), (3,), (3,), (3,), (3,), (3,), (3,), (3,), (3,), (3,), (3,), (3,), (3,), (3,), (3,), (3,), (3,), (3,), (3,), (3,), (3,), (3,), (3,), (3,), (3,), (3,), (3,), (3,), (3,), (3,), (3,), (3,), (3,), (3,), (3,), (3,), (3,), (3,), (3,), (3,), (3,), (3,), (3,), (3,), (3,), (3,), (3,), (3,), (3,), (3,), (3,), (3,), (3,), (3,), (3,), (3,), (3,), (3,), (3,), (3,), (3,), (3,), (3,

In [ ]:
cursor.execute("SELECT userId,rating,time FROM rating WHERE rating='4.0' LIMIT 20 ")
row = cursor.fetchall()
row

[(1, 4.0, '2004-09-10 03:08:54'),
 (1, 4.0, '2005-04-02 23:46:13'),
 (1, 4.0, '2005-04-02 23:35:40'),
 (1, 4.0, '2005-04-02 23:33:46'),
 (1, 4.0, '2005-04-02 23:31:43'),
 (1, 4.0, '2005-04-02 23:32:47'),
 (1, 4.0, '2005-04-02 23:33:18'),
 (1, 4.0, '2005-04-02 23:30:03'),
 (1, 4.0, '2005-04-02 23:44:40'),
 (1, 4.0, '2004-09-10 03:07:45'),
 (1, 4.0, '2005-04-02 23:44:13'),
 (1, 4.0, '2005-04-02 23:48:21'),
 (1, 4.0, '2005-04-02 23:29:20'),
 (1, 4.0, '2004-09-10 03:12:57'),
 (1, 4.0, '2004-09-10 03:14:42'),
 (1, 4.0, '2004-09-10 03:13:14'),
 (1, 4.0, '2005-04-02 23:43:21'),
 (1, 4.0, '2005-04-02 23:43:02'),
 (1, 4.0, '2004-09-10 03:13:14'),
 (1, 4.0, '2005-04-02 23:44:00')]

In [ ]:
text='''
SELECT userId, rating,
CASE
    WHEN rating > 3 THEN 'The rating is greater than 3'
    WHEN rating = 3 THEN 'The rating is 3'
    ELSE 'The rating is under 3'
END AS QuantityText
FROM rating
LIMIT 10
'''
cursor.execute(text)
row = cursor.fetchall()
row

[(1, 3.5, 'The rating is greater than 3'),
 (1, 3.5, 'The rating is greater than 3'),
 (1, 3.5, 'The rating is greater than 3'),
 (1, 3.5, 'The rating is greater than 3'),
 (1, 3.5, 'The rating is greater than 3'),
 (1, 3.5, 'The rating is greater than 3'),
 (1, 4.0, 'The rating is greater than 3'),
 (1, 4.0, 'The rating is greater than 3'),
 (1, 4.0, 'The rating is greater than 3'),
 (1, 4.0, 'The rating is greater than 3')]

In [ ]:
text='''
SELECT movieId, rating
FROM rating 
GROUP BY movieId
HAVING rating>4.0
LIMIT 20
'''
cursor.execute(text)
row = cursor.fetchall()
row

[(6, 5.0),
 (18, 4.5),
 (25, 5.0),
 (32, 5.0),
 (36, 5.0),
 (38, 4.5),
 (50, 4.5),
 (53, 5.0),
 (58, 4.5),
 (63, 5.0),
 (69, 4.5),
 (70, 4.5),
 (80, 5.0),
 (83, 5.0),
 (90, 5.0),
 (97, 4.5),
 (107, 4.5),
 (108, 5.0),
 (111, 5.0),
 (116, 5.0)]

The main difference between WHERE and HAVING clause comes when used together with GROUP BY clause, In that case WHERE is used to filter rows (pre-filter) before grouping and HAVING is used to exclude records after grouping. (post-filter)

In [ ]:
text='''
SELECT movieId, COUNT(userId) as total
FROM rating 
GROUP BY movieId
HAVING total>100
LIMIT 20
'''
cursor.execute(text)
row = cursor.fetchall()
row

[(1, 49695),
 (2, 22243),
 (3, 12735),
 (4, 2756),
 (5, 12161),
 (6, 23899),
 (7, 12961),
 (8, 1415),
 (9, 3960),
 (10, 29005),
 (11, 18162),
 (12, 3845),
 (13, 1461),
 (14, 6022),
 (15, 2910),
 (16, 17394),
 (17, 20667),
 (18, 5203),
 (19, 20938),
 (20, 4084)]

Use Dask

## world.sql

In [ ]:
import sqlite3
datapath='/content/drive/My Drive/Data/SQL/world.sql'
conn = sqlite3.connect(datapath)
cursor = conn.cursor()

cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
table_name=cursor.fetchall()
table_name=table_name[0][0]
print('table_name: ',table_name)

In [ ]:
pd.read_sql('SELECT * FROM world',conn)

In [ ]:
import pyodbc 
datapath='/content/drive/My Drive/Data/SQL/world.sql'
conn = pyodbc.connect("Driver={SQL Server};"
                      "Server=/content/drive/My Drive/Data/SQL/;"
                      "Database=world;"
                      "Trusted_Connection=yes;")
cursor = conn.cursor()
cursor.execute('SELECT * FROM Table')

for row in cursor:
    print('row = %r' % (row,))

Error: ignored

In [ ]:
!pip install pymssql

     |████████████████████████████████| 1.3MB 2.6MB/s 


In [ ]:
import pymssql

conn = pymssql.connect(datapath)
cursor = conn.cursor()

cursor.execute('SELECT * FROM Table')

for row in cursor:
    print('row = %r' % (row,))

In [ ]:
!pip install MySQL-python

     |████████████████████████████████| 112kB 2.8MB/s 
ERROR: Command errored out with exit status 1: python setup.py egg_info Check the logs for full command output.


In [ ]:
import MySQLdb

conn = MySQLdb.connect(datapath)
cursor = conn.cursor()
cursor.execute('SELECT * FROM Table')

for row in cursor:
    print('row = %r' % (row,))

ModuleNotFoundError: ignored

## Use pyodbc

In [ ]:
!apt install unixodbc-dev
!pip install pyodbc

In [ ]:
import pyodbc 
cnxn = pyodbc.connect("Driver={SQL Server};"
                      "Server=server_name;"
                      "Database=world;"
                      "Trusted_Connection=yes;")


cursor = cnxn.cursor()
cursor.execute('SELECT * FROM Table')

for row in cursor:
    print('row = %r' % (row,))

In [ ]:
import pandas as pd
import pyodbc 
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=RON\SQLEXPRESS;'
                      'Database=TestDB;'
                      'Trusted_Connection=yes;')

cursor = conn.cursor()

sql_query = pd.read_sql_query('SELECT * FROM TestDB.dbo.Person',conn)
print(sql_query)
print(type(sql_query))

## Microsoft sql

In [ ]:
import pymssql

conn = pymssql.connect(server=server, user=user, password=password, database=db)
cursor = conn.cursor()

cursor.execute("SELECT COUNT(MemberID) as count FROM Members WHERE id = 1")
row = cursor.fetchone()

conn.close()

print(row)

## Postgres

In [ ]:
import psycopg2

conn = psycopg2.connect(database=db, user=user, password=password, host=host, port="5432")
cursor = conn.cursor()

cursor.execute('SELECT COUNT(MemberID) as count FROM Members WHERE id = 1')
row = cursor.fetchone()

conn.close()

print(row)

## mysql

In [ ]:
import MySQLdb

conn = MySQLdb.connect(host=host, user=user, passwd=passwd, db=db)
cursor = conn.cursor()

cursor.execute('SELECT COUNT(MemberID) as count FROM Members WHERE id = 1')
row = cursor.fetchone()

conn.close()

print(row)